In [8]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sportsreference.nba.roster import Player

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Build player bio dataframe and merge with master 1997 - 2020 df

### Steps:
  - figure out sportsreference api
  - get all player ids in master dataset
  - pull bio data for all player ids
  - save bio df
  - combine bio and stats dfs on multindex of name and player_id
  - save master df
  
 ### Had to fix error of missing 2000 year data (messy in the middle)


In [5]:
df = pd.read_csv('../bballref_data/combo_1997_to_2020_advanced_shooting_pbp_per_poss.csv', index_col=0)
df.shape
df.head()

(11162, 88)

,player_id,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,...,fta_per_poss,gs,off_rtg,orb_per_poss,pf_per_poss,pts_per_poss,stl_per_poss,tov_per_poss,trb_per_poss,drawn_offensive
player,,,,,,,,,,,,,,,,,,,,,
Mahmoud Abdul-Rauf 1997,abdulma02,PG,27,SAC,75,2131,13.2,0.524,0.266,0.147,...,3.4,51,105.0,0.4,4.3,25.5,1.4,2.9,3.0,NaN
Shareef Abdur-Rahim 1997,abdursh01,PF,20,VAN,80,2802,17.4,0.518,0.022,0.428,...,10.1,71,102.0,4.2,3.9,28.9,1.5,4.4,10.7,NaN
Rafael Addison 1997,addisra01,SF,32,CHH,41,355,10.6,0.476,0.164,0.230,...,4.3,3,103.0,2.9,7.9,19.5,1.2,2.6,6.9,NaN
Cory Alexander 1997,alexaco01,PG,23,SAS,80,1454,14.3,0.528,0.514,0.263,...,4.9,6,101.0,1.1,5.6,21.8,3.1,5.5,4.7,NaN
Jerome Allen 1997,allenje01,SG,24,TOT,76,943,8.7,0.451,0.421,0.326,...,4.1,1,95.0,1.4,4.8,12.9,1.8,3.9,5.5,NaN


### Get unique player_ids

In [6]:
player_ids = df['player_id'].unique()
len(player_ids)

2246

### create bio df

In [9]:
## figuring out api

cols=['player_id', 'height', 'weight', 'salary', 'nationality']

df_bio = pd.DataFrame(columns=['player_id', 'height', 'weight', 'salary', 'nationality'])

for pid in player_ids[0:5]:
    player = Player(pid)
    df_player = player.dataframe
    df_player.index = df_player.index.get_level_values(0)
    df_player.index = df_player.index.map(lambda x: player.name + ' ' + x[0:2] + x[-2:])
    df_player = df_player[cols]
    df_bio = df_bio.append(df_player)

df_bio.index.rename('player', inplace=True)

## Function to get df from api

In [28]:
def build_player_df(player_ids, cols, filename):
    df_all = pd.DataFrame(columns=cols)
    for pid in player_ids:
        player = Player(pid)
        df_player = player.dataframe
        df_player.index = df_player.index.get_level_values(0) # comes with multiindex
        df_player.index = df_player.index.map(lambda x: player.name + ' ' + x[0:2] + x[-2:] if x != '1999-00' else player.name + ' 2000')
        df_player = df_player[cols]
        df_all = df_all.append(df_player)

    df_all.index.rename('player', inplace=True)
    df_all['year'] = df_all.index.map(lambda x: x[-4:])
    print(df_all.head())
    print(df_all.columns)
    
    df_all.to_csv(filename)
    return df_all

# cols=['player_id', 'height', 'weight', 'salary', 'nationality']
# filename = 'BIO_TEST2.csv'
# df = build_player_df(player_ids[0:5], cols, filename)


## Pull all bio data and write out

In [132]:
%%time

cols=['player_id', 'height', 'weight', 'salary', 'nationality']
filename = 'player_bio_data.csv'
df = build_player_df(player_ids, cols, filename)

## took 46 min...

                         player_id height weight   salary  \
player                                                      
Mahmoud Abdul-Rauf 1991  abdulma02    6-1    162  1660000   
Mahmoud Abdul-Rauf 1992  abdulma02    6-1    162  2008000   
Mahmoud Abdul-Rauf 1993  abdulma02    6-1    162  2358000   
Mahmoud Abdul-Rauf 1994  abdulma02    6-1    162  1825000   
Mahmoud Abdul-Rauf 1995  abdulma02    6-1    162  2200000   

                                      nationality  year  
player                                                   
Mahmoud Abdul-Rauf 1991  United States of America  1991  
Mahmoud Abdul-Rauf 1992  United States of America  1992  
Mahmoud Abdul-Rauf 1993  United States of America  1993  
Mahmoud Abdul-Rauf 1994  United States of America  1994  
Mahmoud Abdul-Rauf 1995  United States of America  1995  
Index(['player_id', 'height', 'weight', 'salary', 'nationality', 'year'], dtype='object')
Wall time: 46min 45s


# Merge on `player, player_id, year`

In [43]:
df_master = pd.read_csv('../bballref_data/combo_1997_to_2020_advanced_shooting_pbp_per_poss.csv', index_col=0)
df_bio = pd.read_csv('../bballref_data/player_bio_data.csv', index_col=0)

In [36]:
## quick fix for earlier 1990-00 parsing error

df_bio[df_bio.index.str.contains('Allen Iverson')]
df_bio.index = df_bio.index.map(lambda x: x if x[-4:] != '1900' else x[0:-4] + '2000')
df_bio[df_bio.index.str.contains('Allen Iverson')]

df_bio.loc[:, 'year'] = df_bio['year'].apply(lambda x: x if x != '1900' else '2000' )

df_bio[df_bio.index.str.contains('Allen Iverson')]

,player_id,height,weight,salary,nationality,year
player,,,,,,
Allen Iverson 1997,iversal01,6-0,165,2267000.0,United States of America,1997
Allen Iverson 1998,iversal01,6-0,165,3128640.0,United States of America,1998
Allen Iverson 1999,iversal01,6-0,165,3537000.0,United States of America,1999
Allen Iverson 2000,iversal01,6-0,165,9000000.0,United States of America,2000
Allen Iverson 2001,iversal01,6-0,165,10130000.0,United States of America,2001
Allen Iverson 2002,iversal01,6-0,165,11250000.0,United States of America,2002
Allen Iverson 2003,iversal01,6-0,165,12375000.0,United States of America,2003
Allen Iverson 2004,iversal01,6-0,165,13500000.0,United States of America,2004
Allen Iverson 2005,iversal01,6-0,165,14625000.0,United States of America,2005


In [38]:
df_bio.to_csv('../bballref_data/player_bio_data.csv')

In [45]:
## cant merge bc dtypes..
## fix dtype (drop unneccessary rows also)

# df_bio['year'].dtype
# df_master['year'].dtype

# df_bio['player_id'].dtype
# df_master['player_id'].dtype

df_bio = df_bio[df_bio['year'] != 'Caer'].copy()
df_bio.loc[:, 'year'] = df_bio['year'].astype(int)

In [46]:
df_bio.shape
df_master.shape

(14068, 6)

(11162, 88)

In [47]:
df_bio['year'].unique()

array([1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 2001, 1999, 2000,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2010, 1987, 2009, 2011,
       2012, 2013, 2014, 1988, 1989, 1990, 1985, 1986, 2015, 2016, 1982,
       1983, 1984, 1981, 2017, 1979, 1980, 1977, 1978, 2018, 2019, 2020])

In [19]:
ai = Player('iversal01').dataframe
ai.salary

1996-97      2267000
1997-98      3128640
1998-99      3537000
1999-00      9000000
2000-01     10130000
2001-02     11250000
2002-03     12375000
2003-04     13500000
2004-05     14625000
2005-06     16453125
2006-07     17184375
2007-08     19012500
2008-09     20840625
2009-10      1029794
Career     154494445
Name: salary, dtype: int64

In [18]:
df_bio[df_bio.index.str.contains('Allen Iverson')]

,player_id,height,weight,salary,nationality,year
player,,,,,,
Allen Iverson 1997,iversal01,6-0,165,2267000.0,United States of America,1997
Allen Iverson 1998,iversal01,6-0,165,3128640.0,United States of America,1998
Allen Iverson 1999,iversal01,6-0,165,3537000.0,United States of America,1999
Allen Iverson 1900,iversal01,6-0,165,9000000.0,United States of America,1900
Allen Iverson 2001,iversal01,6-0,165,10130000.0,United States of America,2001
Allen Iverson 2002,iversal01,6-0,165,11250000.0,United States of America,2002
Allen Iverson 2003,iversal01,6-0,165,12375000.0,United States of America,2003
Allen Iverson 2004,iversal01,6-0,165,13500000.0,United States of America,2004
Allen Iverson 2005,iversal01,6-0,165,14625000.0,United States of America,2005


In [48]:
df_all = pd.merge(df_master, df_bio, on=['player_id', 'year'], left_index=True)
df_all.shape
df_all.iloc[0]
df_all.head()

(11162, 92)

player_id                         abdulma02
pos                                      PG
age                                      27
team_id                                 SAC
g                                        75
                             ...           
drawn_offensive                         NaN
height                                  6-1
weight                                  162
salary                              3.1e+06
nationality        United States of America
Name: Mahmoud Abdul-Rauf 1997, Length: 92, dtype: object

,player_id,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,...,pf_per_poss,pts_per_poss,stl_per_poss,tov_per_poss,trb_per_poss,drawn_offensive,height,weight,salary,nationality
player,,,,,,,,,,,,,,,,,,,,,
Mahmoud Abdul-Rauf 1997,abdulma02,PG,27,SAC,75,2131,13.2,0.524,0.266,0.147,...,4.3,25.5,1.4,2.9,3.0,NaN,6-1,162,3100000.0,United States of America
Shareef Abdur-Rahim 1997,abdursh01,PF,20,VAN,80,2802,17.4,0.518,0.022,0.428,...,3.9,28.9,1.5,4.4,10.7,NaN,6-9,225,2186000.0,United States of America
Rafael Addison 1997,addisra01,SF,32,CHH,41,355,10.6,0.476,0.164,0.230,...,7.9,19.5,1.2,2.6,6.9,NaN,6-7,215,290000.0,United States of America
Cory Alexander 1997,alexaco01,PG,23,SAS,80,1454,14.3,0.528,0.514,0.263,...,5.6,21.8,3.1,5.5,4.7,NaN,6-1,185,521000.0,United States of America
Jerome Allen 1997,allenje01,SG,24,TOT,76,943,8.7,0.451,0.421,0.326,...,4.8,12.9,1.8,3.9,5.5,NaN,6-4,184,247500.0,United States of America


## Write out master df

In [50]:
df_all.to_csv('../bballref_data/master_nba_data.csv')

## WORKSHEET BELOW

In [131]:
df1 = pd.read_csv('BIO_TEST2.csv', index_col=0)
df1.tail()

,player_id,height,weight,salary,nationality,year
player,,,,,,
Jerome Allen 1996,allenje01,6-4,184,200000.0,United States of America,1996
Jerome Allen 1997,allenje01,6-4,184,247500.0,United States of America,1997
Jerome Allen Caer,allenje01,6-4,184,992500.0,United States of America,Caer
Jerome Allen 1998,allenje01,6-4,184,35000.0,United States of America,1998
Jerome Allen 1900,allenje01,6-4,184,510000.0,United States of America,1900


In [126]:
df

,player_id,height,weight,salary,nationality,year
player,,,,,,
Mahmoud Abdul-Rauf 1991,abdulma02,6-1,162,1660000,United States of America,1991
Mahmoud Abdul-Rauf 1992,abdulma02,6-1,162,2008000,United States of America,1992
Mahmoud Abdul-Rauf 1993,abdulma02,6-1,162,2358000,United States of America,1993
Mahmoud Abdul-Rauf 1994,abdulma02,6-1,162,1825000,United States of America,1994
Mahmoud Abdul-Rauf 1995,abdulma02,6-1,162,2200000,United States of America,1995
Mahmoud Abdul-Rauf 1996,abdulma02,6-1,162,2600000,United States of America,1996
Mahmoud Abdul-Rauf 1997,abdulma02,6-1,162,3100000,United States of America,1997
Mahmoud Abdul-Rauf 1998,abdulma02,6-1,162,3300000,United States of America,1998
Mahmoud Abdul-Rauf 2001,abdulma02,6-1,162,798500,United States of America,2001


In [22]:
from sportsreference.nba.roster import Player

for pid in player_ids[0:5]:
    player = Player(pid)
    print(player.height)
    print(player.weight)
    print()
#     print(pid)

6-1
162
6-9
225
6-7
215
6-1
185
6-4
184


In [27]:
for pid in player_ids[0:5]:
    player = Player(pid)
    
    print(player.name)
    print(player('2020').points)
    print(player.height)
    print(player.weight)
    print(player.college)
    print()


Mahmoud Abdul-Rauf
8553
6-1
162


AttributeError: 'Player' object has no attribute 'college'

In [29]:
mj = Player('jordami01').dataframe
mj.head()

,and_ones,assist_percentage,assists,block_percentage,blocking_fouls,blocks,box_plus_minus,center_percentage,defensive_box_plus_minus,defensive_rebound_percentage,...,turnovers,two_point_attempts,two_point_percentage,two_pointers,two_pointers_assisted_percentage,usage_percentage,value_over_replacement_player,weight,win_shares,win_shares_per_48_minutes
1984-85,None,25.3,481.0,1.3,None,69.0,7.3,0,1.5,13.2,...,291.0,1573.0,0.526,828.0,None,29.8,7.4,198,14.0,0.213
1985-86,None,21.7,53.0,2.7,None,21.0,9.7,0,2.2,10.7,...,45.0,310.0,0.474,147.0,None,38.6,1.3,198,1.5,0.160
1986-87,None,22.2,377.0,2.3,None,125.0,10.8,0,2.4,9.3,...,272.0,2213.0,0.491,1086.0,None,38.3,10.6,198,16.9,0.247
1987-88,None,27.0,485.0,2.4,None,131.0,13.0,0,4.2,10.7,...,252.0,1945.0,0.546,1062.0,None,34.1,12.5,198,21.2,0.308
1988-89,None,34.7,650.0,1.2,None,65.0,11.9,0,3.4,17.3,...,290.0,1697.0,0.553,939.0,None,32.1,11.4,198,19.8,0.292


In [30]:
mj.salary

1984-85      550000.0
1985-86      630000.0
1986-87           NaN
1987-88      845000.0
1988-89     2000000.0
1989-90     2500000.0
1990-91     2500000.0
1991-92     3250000.0
1992-93     4000000.0
1994-95     3850000.0
1995-96     3850000.0
1996-97    30140000.0
1997-98    33140000.0
2001-02     1000000.0
2002-03     1030000.0
Career     93285000.0
1993-94     4000000.0
Name: salary, dtype: float64

In [79]:
mj = Player('jordami01').dataframe

# mj.index = [year for year in mj.index]
mj.head()

,and_ones,assist_percentage,assists,block_percentage,blocking_fouls,blocks,box_plus_minus,center_percentage,defensive_box_plus_minus,defensive_rebound_percentage,...,turnovers,two_point_attempts,two_point_percentage,two_pointers,two_pointers_assisted_percentage,usage_percentage,value_over_replacement_player,weight,win_shares,win_shares_per_48_minutes
1984-85,None,25.3,481.0,1.3,None,69.0,7.3,0,1.5,13.2,...,291.0,1573.0,0.526,828.0,None,29.8,7.4,198,14.0,0.213
1985-86,None,21.7,53.0,2.7,None,21.0,9.7,0,2.2,10.7,...,45.0,310.0,0.474,147.0,None,38.6,1.3,198,1.5,0.160
1986-87,None,22.2,377.0,2.3,None,125.0,10.8,0,2.4,9.3,...,272.0,2213.0,0.491,1086.0,None,38.3,10.6,198,16.9,0.247
1987-88,None,27.0,485.0,2.4,None,131.0,13.0,0,4.2,10.7,...,252.0,1945.0,0.546,1062.0,None,34.1,12.5,198,21.2,0.308
1988-89,None,34.7,650.0,1.2,None,65.0,11.9,0,3.4,17.3,...,290.0,1697.0,0.553,939.0,None,32.1,11.4,198,19.8,0.292


In [80]:
mj.index = mj.index.get_level_values(0)
mj.index
mj.head()


Index(['1984-85', '1985-86', '1986-87', '1987-88', '1988-89', '1989-90',
       '1990-91', '1991-92', '1992-93', '1994-95', '1995-96', '1996-97',
       '1997-98', '2001-02', '2002-03', 'Career', '1993-94'],
      dtype='object')

,and_ones,assist_percentage,assists,block_percentage,blocking_fouls,blocks,box_plus_minus,center_percentage,defensive_box_plus_minus,defensive_rebound_percentage,...,turnovers,two_point_attempts,two_point_percentage,two_pointers,two_pointers_assisted_percentage,usage_percentage,value_over_replacement_player,weight,win_shares,win_shares_per_48_minutes
1984-85,None,25.3,481.0,1.3,None,69.0,7.3,0,1.5,13.2,...,291.0,1573.0,0.526,828.0,None,29.8,7.4,198,14.0,0.213
1985-86,None,21.7,53.0,2.7,None,21.0,9.7,0,2.2,10.7,...,45.0,310.0,0.474,147.0,None,38.6,1.3,198,1.5,0.160
1986-87,None,22.2,377.0,2.3,None,125.0,10.8,0,2.4,9.3,...,272.0,2213.0,0.491,1086.0,None,38.3,10.6,198,16.9,0.247
1987-88,None,27.0,485.0,2.4,None,131.0,13.0,0,4.2,10.7,...,252.0,1945.0,0.546,1062.0,None,34.1,12.5,198,21.2,0.308
1988-89,None,34.7,650.0,1.2,None,65.0,11.9,0,3.4,17.3,...,290.0,1697.0,0.553,939.0,None,32.1,11.4,198,19.8,0.292


In [85]:
mj.index = mj.index.map(lambda x: Player('jordami01').name + ' ' x[0:2] + x[-2:])
mj.head()

,and_ones,assist_percentage,assists,block_percentage,blocking_fouls,blocks,box_plus_minus,center_percentage,defensive_box_plus_minus,defensive_rebound_percentage,...,turnovers,two_point_attempts,two_point_percentage,two_pointers,two_pointers_assisted_percentage,usage_percentage,value_over_replacement_player,weight,win_shares,win_shares_per_48_minutes
Michael Jordan1985,None,25.3,481.0,1.3,None,69.0,7.3,0,1.5,13.2,...,291.0,1573.0,0.526,828.0,None,29.8,7.4,198,14.0,0.213
Michael Jordan1986,None,21.7,53.0,2.7,None,21.0,9.7,0,2.2,10.7,...,45.0,310.0,0.474,147.0,None,38.6,1.3,198,1.5,0.160
Michael Jordan1987,None,22.2,377.0,2.3,None,125.0,10.8,0,2.4,9.3,...,272.0,2213.0,0.491,1086.0,None,38.3,10.6,198,16.9,0.247
Michael Jordan1988,None,27.0,485.0,2.4,None,131.0,13.0,0,4.2,10.7,...,252.0,1945.0,0.546,1062.0,None,34.1,12.5,198,21.2,0.308
Michael Jordan1989,None,34.7,650.0,1.2,None,65.0,11.9,0,3.4,17.3,...,290.0,1697.0,0.553,939.0,None,32.1,11.4,198,19.8,0.292


In [70]:
df[df.index.str.contains('Michael Jordan')]['ast_pct']

player
Michael Jordan* 1997    21.2
Michael Jordan* 1998    18.0
Michael Jordan* 2002    30.8
Michael Jordan* 2003    21.1
Name: ast_pct, dtype: float64

In [77]:
mj[['height', 'weight', 'salary', 'nationality', 'player_id']]

,height,weight,salary,nationality,player_id
1985,6-6,198,550000.0,United States of America,jordami01
1986,6-6,198,630000.0,United States of America,jordami01
1987,6-6,198,NaN,United States of America,jordami01
1988,6-6,198,845000.0,United States of America,jordami01
1989,6-6,198,2000000.0,United States of America,jordami01
1990,6-6,198,2500000.0,United States of America,jordami01
1991,6-6,198,2500000.0,United States of America,jordami01
1992,6-6,198,3250000.0,United States of America,jordami01
1993,6-6,198,4000000.0,United States of America,jordami01
1995,6-6,198,3850000.0,United States of America,jordami01


In [115]:
sorted(list(df_mj.columns))

['and_ones',
 'assist_percentage',
 'assists',
 'block_percentage',
 'blocking_fouls',
 'blocks',
 'box_plus_minus',
 'center_percentage',
 'defensive_box_plus_minus',
 'defensive_rebound_percentage',
 'defensive_rebounds',
 'defensive_win_shares',
 'dunks',
 'effective_field_goal_percentage',
 'field_goal_attempts',
 'field_goal_perc_sixteen_foot_plus_two_pointers',
 'field_goal_perc_ten_to_sixteen_feet',
 'field_goal_perc_three_to_ten_feet',
 'field_goal_perc_zero_to_three_feet',
 'field_goal_percentage',
 'field_goals',
 'free_throw_attempt_rate',
 'free_throw_attempts',
 'free_throw_percentage',
 'free_throws',
 'games_played',
 'games_started',
 'half_court_heaves',
 'half_court_heaves_made',
 'height',
 'lost_ball_turnovers',
 'minutes_played',
 'nationality',
 'net_plus_minus',
 'offensive_box_plus_minus',
 'offensive_fouls',
 'offensive_rebound_percentage',
 'offensive_rebounds',
 'offensive_win_shares',
 'on_court_plus_minus',
 'other_turnovers',
 'passing_turnovers',
 'percen

In [89]:
## Need to merge on 

mj = Player('jordami01')
df_mj = mj.dataframe
df_mj.index = df_mj.index.get_level_values(0)
df_mj.index = df_mj.index.map(lambda x: mj.name + ' ' + x[0:2] + x[-2:])
df_mj[['height', 'weight', 'salary', 'nationality', 'player_id']]

,height,weight,salary,nationality,player_id
Michael Jordan 1985,6-6,198,550000.0,United States of America,jordami01
Michael Jordan 1986,6-6,198,630000.0,United States of America,jordami01
Michael Jordan 1987,6-6,198,NaN,United States of America,jordami01
Michael Jordan 1988,6-6,198,845000.0,United States of America,jordami01
Michael Jordan 1989,6-6,198,2000000.0,United States of America,jordami01
Michael Jordan 1990,6-6,198,2500000.0,United States of America,jordami01
Michael Jordan 1991,6-6,198,2500000.0,United States of America,jordami01
Michael Jordan 1992,6-6,198,3250000.0,United States of America,jordami01
Michael Jordan 1993,6-6,198,4000000.0,United States of America,jordami01
Michael Jordan 1995,6-6,198,3850000.0,United States of America,jordami01


In [ ]:
## will create new 